In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Step 1: Data Collection (User-Item Interaction Matrix)
data = {
    'User': ['User1', 'User2', 'User3', 'User4'],
    'Movie A': [5, 4, 0, 0],
    'Movie B': [4, 0, 0, 0],
    'Movie C': [0, 0, 5, 4],
    'Movie D': [0, 3, 4, 5],
    'Movie E': [0, 0, 0, 5]
}
ratings = pd.DataFrame(data)
ratings.set_index('User', inplace=True)

# Step 2: Data Processing (Calculate Item Similarities)
# Transpose the matrix for item-based collaborative filtering
item_similarity = cosine_similarity(ratings.T)
similarity_df = pd.DataFrame(item_similarity, index=ratings.columns, columns=ratings.columns)

# Step 3: Prediction of Preferences (Score Calculation for Each User)
def recommend_items(user, ratings, similarity_df):
    user_ratings = ratings.loc[user]
    scores = {}
    for item in ratings.columns:
        if user_ratings[item] == 0:  # Recommend only for items not rated
            sim_items = similarity_df[item]
            rated_items = user_ratings[user_ratings > 0].index
            score = sum(sim_items[rated_item] * user_ratings[rated_item] for rated_item in rated_items)
            scores[item] = score
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)

# Step 4: Recommendation Generation and Filtering
user_to_recommend = 'User2'
recommendations = recommend_items(user_to_recommend, ratings, similarity_df)

# Step 5: Presentation
print(f"Recommendations for {user_to_recommend}:\n")
for item, score in recommendations:
    print(f"{item}: {score:.2f}")


Recommendations for User2:

Movie B: 3.12
Movie C: 2.65
Movie E: 2.12


In [3]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Step 1: Data Collection (User-Item Interaction Matrix)
data = {
    'User': ['User1', 'User2', 'User3', 'User4'],
    'Movie A': [5, 4, 0, 0],
    'Movie B': [4, 0, 0, 0],
    'Movie C': [0, 0, 5, 4],
    'Movie D': [0, 3, 4, 5],
    'Movie E': [0, 0, 0, 5]
}
ratings = pd.DataFrame(data)
ratings.set_index('User', inplace=True)

# Step 2: Data Processing (Calculate Item Similarities for Collaborative Filtering)
# Transpose the matrix for item-based collaborative filtering
item_similarity = cosine_similarity(ratings.T)
similarity_df = pd.DataFrame(item_similarity, index=ratings.columns, columns=ratings.columns)

# Content-Based Filtering (Step 2A)
# Assume each movie has metadata (e.g., genres)
metadata = {
    'Movie A': [1, 0, 1],  # Action, Comedy, Drama
    'Movie B': [1, 1, 0],  # Action, Comedy, Drama
    'Movie C': [0, 1, 1],  # Action, Comedy, Drama
    'Movie D': [0, 0, 1],  # Action, Comedy, Drama
    'Movie E': [1, 0, 0],  # Action, Comedy, Drama
}
metadata_df = pd.DataFrame(metadata, index=['Action', 'Comedy', 'Drama']).T
content_similarity = cosine_similarity(metadata_df)
content_similarity_df = pd.DataFrame(content_similarity, index=metadata_df.index, columns=metadata_df.index)

# Step 3: Recommendation Techniques
def collaborative_filtering(user, ratings, similarity_df):
    user_ratings = ratings.loc[user]
    scores = {}
    for item in ratings.columns:
        if user_ratings[item] == 0:  # Recommend only for items not rated
            sim_items = similarity_df[item]
            rated_items = user_ratings[user_ratings > 0].index
            score = sum(sim_items[rated_item] * user_ratings[rated_item] for rated_item in rated_items)
            scores[item] = score
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)

def content_based_filtering(user, ratings, content_similarity_df):
    user_ratings = ratings.loc[user]
    scores = {}
    for item in ratings.columns:
        if user_ratings[item] == 0:  # Recommend only for items not rated
            sim_items = content_similarity_df[item]
            rated_items = user_ratings[user_ratings > 0].index
            score = sum(sim_items[rated_item] * user_ratings[rated_item] for rated_item in rated_items)
            scores[item] = score
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)

# Hybrid Recommendation: Combining Collaborative and Content-Based Scores
def hybrid_recommendation(user, ratings, similarity_df, content_similarity_df, alpha=0.5):
    collaborative_scores = dict(collaborative_filtering(user, ratings, similarity_df))
    content_scores = dict(content_based_filtering(user, ratings, content_similarity_df))
    hybrid_scores = {}
    for item in ratings.columns:
        hybrid_scores[item] = alpha * collaborative_scores.get(item, 0) + (1 - alpha) * content_scores.get(item, 0)
    return sorted(hybrid_scores.items(), key=lambda x: x[1], reverse=True)

# Step 4: Generate Recommendations
user_to_recommend = 'User1'
print(f"Collaborative Filtering Recommendations for {user_to_recommend}:")
print(collaborative_filtering(user_to_recommend, ratings, similarity_df))

print(f"\nContent-Based Filtering Recommendations for {user_to_recommend}:")
print(content_based_filtering(user_to_recommend, ratings, content_similarity_df))

print(f"\nHybrid Recommendations for {user_to_recommend}:")
print(hybrid_recommendation(user_to_recommend, ratings, similarity_df, content_similarity_df))


Collaborative Filtering Recommendations for User1:
[('Movie D', 1.3251783128981587), ('Movie C', 0.0), ('Movie E', 0.0)]

Content-Based Filtering Recommendations for User1:
[('Movie E', 6.363961030678928), ('Movie C', 4.499999999999999), ('Movie D', 3.5355339059327373)]

Hybrid Recommendations for User1:
[('Movie E', 3.181980515339464), ('Movie D', 2.430356109415448), ('Movie C', 2.2499999999999996), ('Movie A', 0.0), ('Movie B', 0.0)]
